In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.nn.functional as F

#Importing training set and pick the x, y coordinates and facing angle
dataset_train=pd.read_csv('pos-2.csv')
training_set=dataset_train.iloc[:,1:4].values

#Scaling
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=(0,1))
training_set_scaled=sc.fit_transform(training_set)

#Parameters
duration = 75
input_size = 1
hidden_size = 32
num_layers = 2
output_size = 1
num_epochs = 100

#Creating a data structure with duration of 75
X_train=[]
y_train=[]
for i in range(duration,581):
    X_train.append(training_set_scaled[i-duration:i,0])
    y_train.append(training_set_scaled[i,0])
X_train = np.array(X_train)
y_train = np.array(y_train)
X_train = torch.from_numpy(X_train).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)

class LSTMWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, duration):
        super(LSTMWithAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.layer1 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.layer2 = nn.Linear(hidden_size, output_size)

        self.w = nn.Parameter(torch.Tensor(hidden_size, 1))
        self.b = nn.Parameter(torch.Tensor(duration, 1))
        nn.init.uniform_(self.w)
        nn.init.zeros_(self.b)

    # soft attention layer
    def attention_net(self, x):
        e = torch.tanh(torch.matmul(x, self.w) + self.b)
        alpha = F.softmax(e, dim=1)
        x = x * alpha
        x = torch.sum(x, dim=1)
        return x

    def forward(self, x):
        h = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        h = h.requires_grad_()
        c = c.requires_grad_()
        h = h.detach()
        c = c.detach()
        out, (h_final, c_final) = self.layer1(x, (h, c))
        out = self.attention_net(out)
        out = self.layer2(out)
        return out

model = LSTMWithAttention(input_size=input_size, hidden_size=hidden_size, output_size=output_size, num_layers=num_layers, duration=duration)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

for t in range(num_epochs):
    y_train_pred = model(X_train.unsqueeze(-1))
    loss = criterion(y_train_pred, y_train.unsqueeze(-1))
    if t%5 ==0:
        print("Epoch:", t, " MSE:", loss.item())
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

In [ ]:
# Send in test data and do evaluation
dataset_test=pd.read_csv('pos-1.csv')
testing_set=dataset_test.iloc[:,1:4].values

#Scaling
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler(feature_range=(0,1))
testing_set_scaled=sc.fit_transform(testing_set)

#Creating a data structure with duration of 75
X_test=[]
y_test=[]
for i in range(duration,289):
    X_test.append(training_set_scaled[i-duration:i,0])
    y_test.append(training_set_scaled[i,0])
X_test = np.array(X_test)
y_test = np.array(y_test)
X_test = torch.from_numpy(X_test).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

y_test_pred = model(X_test.unsqueeze(-1))
loss = criterion(y_test_pred, y_test.unsqueeze(-1))
print("Test MSE: ", loss.item())